# Testing different representations and their respective accuracies on CIFAR10

In [1]:
%%capture
!pip install torch torchvision

Baseline MLP

In [5]:
import torch.nn as nn
from torch.nn.functional import cosine_similarity
import torchvision.transforms.functional as TF
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

In [6]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [7]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Define transformation to convert CIFAR-10 images to grayscale
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load CIFAR-10 dataset
train_dataset_mlp = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset_mlp = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Define data loaders
batch_size = 128

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Data augmentation
def get_transforms(means, stds):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(means, stds),
    ])
    return transform

# Calculate the mean and std of the subset dataset
def get_mean_std(dataset):
    """Compute the mean and std value of dataset."""
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(3):
            mean[i] += inputs[:, i, :, :].mean()
            std[i] += inputs[:, i, :, :].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

In [6]:
# Calculate the mean and std of the subset dataset
train_means_mlp, train_stds_mlp = get_mean_std(train_dataset_mlp)
print(f'Means: {train_means_mlp}')
print(f'Standard deviations: {train_stds_mlp}\n')
val_means_mlp, val_stds_mlp = get_mean_std(test_dataset_mlp)
print(f'Validation means: {val_means_mlp}')
print(f'Validation standard deviations: {val_stds_mlp}')

==> Computing mean and std..
Means: tensor([0.4914, 0.4822, 0.4465])
Standard deviations: tensor([0.1953, 0.1925, 0.1942])

==> Computing mean and std..
Validation means: tensor([0.4942, 0.4851, 0.4504])
Validation standard deviations: tensor([0.1949, 0.1922, 0.1944])


In [7]:
%%capture
train_dataset_mlp = datasets.CIFAR10(root='./data', train=True, download=True, transform=get_transforms(train_means_mlp,train_stds_mlp))
val_dataset_mlp = datasets.CIFAR10(root='./data', train=False, download=True, transform=get_transforms(val_means_mlp,val_stds_mlp))

# Data Loader
train_loader_mlp = DataLoader(train_dataset_mlp, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader_mlp = DataLoader(val_dataset_mlp, batch_size=batch_size, shuffle=False, num_workers=2)

In [8]:
device = "cuda"
input_size = 224 * 224 * 3  # Size of CIFAR-10 images after flattening
hidden_size = 256
output_size = 10  # Number of classes in CIFAR-10
mlp_model = MLP(input_size, hidden_size, output_size)
mlp_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

num_epochs = 10

In [12]:
for epoch in range(num_epochs):
    mlp_model.train()
    total_loss = 0
    correct = 0
    total_samples = 0

    for i, data in enumerate(tqdm(train_loader_mlp, desc=f'Epoch {epoch + 1}/{num_epochs}')):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = mlp_model(images.view(images.size(0), -1))

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total_samples += labels.size(0)

    average_loss = total_loss / len(train_loader_mlp)
    accuracy = correct / total_samples

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}, Accuracy: {accuracy}')

print('Training finished.')

Epoch 1/10: 100%|██████████| 391/391 [01:54<00:00,  3.42it/s]


Epoch 1/10, Loss: 17.21063083212089, Accuracy: 0.30888


Epoch 2/10: 100%|██████████| 391/391 [01:37<00:00,  4.02it/s]


Epoch 2/10, Loss: 3.088271303859818, Accuracy: 0.40726


Epoch 3/10: 100%|██████████| 391/391 [01:50<00:00,  3.54it/s]


...

Epoch 8/10, Loss: 2.0605919226965943, Accuracy: 0.4319


Epoch 9/10: 100%|██████████| 391/391 [01:30<00:00,  4.33it/s]


Epoch 9/10, Loss: 2.1912501605270465, Accuracy: 0.42092


Epoch 10/10: 100%|██████████| 391/391 [01:29<00:00,  4.34it/s]

Epoch 10/10, Loss: 2.3590534202887885, Accuracy: 0.41556
Training finished.


In [13]:
mlp_model.eval()
correct = 0
total_samples = 0

with torch.no_grad():
    for data in tqdm(test_loader_mlp, desc='Testing'):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        outputs = mlp_model(images.view(images.size(0), -1))
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total_samples += labels.size(0)

accuracy = correct / total_samples
print(f'Test Accuracy: {accuracy}')

Testing: 100%|██████████| 79/79 [00:17<00:00,  4.47it/s]

Test Accuracy: 0.3577


Loading CIFAR10 again

In [1]:
%%capture
!pip install scikit-learn

In [2]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from torch.utils.data import TensorDataset

In [3]:
%%capture
from sklearn.datasets import fetch_openml

# Fetch CIFAR-10 data
cifar10 = fetch_openml('CIFAR_10_small', version=1)

# Extract images and labels
images = cifar10.data
labels = cifar10.target

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [10]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

In [11]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Apply PCA
pca = PCA(n_components=512)  # You can adjust the number of components
train_images_pca = pca.fit_transform(train_images)
test_images_pca = pca.transform(test_images)

In [27]:
# Apply t-SNE (Optional: First reduce dimensionality with PCA for faster processing)
tsne = TSNE(n_components=3, perplexity=50.0, n_iter=1000)
train_images_tsne = tsne.fit_transform(pca.transform(train_images))  # Using PCA reduced data
test_images_tsne = tsne.fit_transform(pca.transform(test_images))    # Using PCA reduced data

In [ ]:
# Apply t-SNE (Optional: First reduce dimensionality withOUT PCA)
tsne = TSNE(n_components=3, perplexity=50.0, n_iter=1000)
train_images_tsne = tsne.fit_transform(train_images)
test_images_tsne = tsne.fit_transform(test_images)

In [28]:
from torch.utils.data import TensorDataset, DataLoader
import torch

# Convert pandas Series to numpy arrays
train_labels_np = train_labels.astype(int).to_numpy()
test_labels_np = test_labels.astype(int).to_numpy()

# Convert to PyTorch tensors
train_dataset_pca = TensorDataset(torch.tensor(train_images_pca, dtype=torch.float32), torch.tensor(train_labels_np, dtype=torch.long))
test_dataset_pca = TensorDataset(torch.tensor(test_images_pca, dtype=torch.float32), torch.tensor(test_labels_np, dtype=torch.long))

train_loader_pca = DataLoader(train_dataset_pca, batch_size=128, shuffle=True)
test_loader_pca = DataLoader(test_dataset_pca, batch_size=128, shuffle=False)

# Repeat for t-SNE data if you plan to use it
train_dataset_tsne = TensorDataset(torch.tensor(train_images_tsne, dtype=torch.float32), torch.tensor(train_labels_np, dtype=torch.long))
test_dataset_tsne = TensorDataset(torch.tensor(test_images_tsne, dtype=torch.float32), torch.tensor(test_labels_np, dtype=torch.long))

train_loader_tsne = DataLoader(train_dataset_tsne, batch_size=128, shuffle=True)
test_loader_tsne = DataLoader(test_dataset_tsne, batch_size=128, shuffle=False)

In [17]:
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total_samples = 0

    for data, target in train_loader:
        # Move data to device (CPU or GPU)
        data, target = data.to(device), target.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Compute loss
        loss = criterion(output, target)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        correct += (predicted == target).sum().item()
        total_samples += target.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total_samples
    return avg_loss, accuracy


In [18]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for data, target in test_loader:
            # Move data to device
            data, target = data.to(device), target.to(device)

            # Forward pass
            output = model(data)

            # Compute loss
            total_loss += criterion(output, target).item()

            # Calculate accuracy
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == target).sum().item()
            total_samples += target.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total_samples
    return avg_loss, accuracy


PCA pipeline

In [19]:
# Initialize the MLP model for PCA
input_size_pca = 512
mlp_model_pca = MLP(input_size_pca, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model_pca.parameters(), lr=0.001)

# Training the model
print('PCA Experiment\n\n')
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    train_loss, train_accuracy = train_model(mlp_model_pca, train_loader_pca, optimizer, criterion, device)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')

# Evaluating the model
test_loss, test_accuracy = evaluate_model(mlp_model_pca, test_loader_pca, criterion, device)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%')

PCA Experiment




 10%|█         | 1/10 [00:00<00:07,  1.17it/s]

Epoch 1/10, Train Loss: 15.7483, Train Acc: 31.12%


 20%|██        | 2/10 [00:01<00:04,  1.85it/s]

Epoch 2/10, Train Loss: 5.3906, Train Acc: 48.81%


...

Epoch 8/10, Train Loss: 0.5974, Train Acc: 83.22%


 90%|█████████ | 9/10 [00:03<00:00,  3.06it/s]

Epoch 9/10, Train Loss: 0.6479, Train Acc: 82.92%


100%|██████████| 10/10 [00:03<00:00,  2.71it/s]

Epoch 10/10, Train Loss: 0.5797, Train Acc: 84.78%
Test Loss: 6.1763, Test Acc: 39.67%


T-sne pipeline

In [ ]:
# Initialize the MLP model for Tsne
input_size_tsne = 3
mlp_model_tsne = MLP(input_size_tsne, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer_tsne = optim.Adam(mlp_model_tsne.parameters(), lr=0.01)

# Training the model
print('T-sne Experiment\n\n')
num_epochs = 100
for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(mlp_model_tsne, train_loader_tsne, optimizer_tsne, criterion, device)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
models = ['Images', 'CLR embeddings', 'PCA', 'T-sne']
accuracies = [0.3577, 0.4863, 0.3967, 0.2872]

# Set up the bar colors
colors = ['skyblue', 'seagreen', 'salmon', 'plum']

# Plotting the bar chart
plt.figure(figsize=(10, 6))
bars = plt.bar(models, accuracies, color=colors)

# Add the data labels on top of the bars
for bar, accuracy in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{accuracy:.4f}',
             ha='center', va='bottom')

# Title and labels
plt.title('Test Accuracy for our MLP model with different inputs')
plt.ylabel('Test Accuracy')
plt.xticks(rotation=45, ha="right")

# Save the plot as a file and display
file_path = 'model_accuracies_bar_chart.png'
plt.tight_layout()
plt.savefig(file_path)
plt.show()
